In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import statistics as stat

In [ ]:
def HW3 (learning_alg,
         X.train,
         Y.train,
         regular_method,
         M,
         c,
         K,
         Error_Type):
    
    if Error_Type == 'MSE':
        score_type = 'neg_mean_squared_error'
    else:
        score_type = 'neg_mean_absolute_error'
    
    
    params_drop = [.1,.2,.3,.4,.5]
    
    best_Phi = params_drop[0]
    best_error = stat.mean(cross_val_score(learning_alg,X,Y,cv = K,scoring = score_type))
    
    for i in params_drop:
        new_X = dropout()
        error = stat.mean(cross_val_score(learning_alg,new_X,Y,cv = K,scoring = score_type))
    
    
    if robust:
        param_robust = [function(Mat,)
        for δ in param_robust:
            new_X = X+δ
            
    cross_val_score(learning_alg,X,Y,cv = K,scoring = score_type)
    cross_val_score(learning_alg,X,Y,cv = K,scoring = score_type)

In [27]:
statistics.mean([1,2,3])

2